<center><h1>DataFrame Sparklines</h1></center>

An approach to [sparklines](https://en.wikipedia.org/wiki/Sparkline) within pandas dataframes using matplotlib.

In [1]:
import base64
import requests
import numpy as np
import pandas as pd
from time import sleep
from itertools import chain
from cStringIO import StringIO
from datetime import timedelta, date
from IPython.display import display, HTML

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Turn off the max column width so the HTML 
# image tags don't get truncated 
pd.set_option('display.max_colwidth', -1)

# Turning off the max column will display all the data in
# our arrays so limit the number of element to display
pd.set_option('display.max_seq_items', 3)

In [3]:
def sparkline(data, figsize=(4, 0.25), **kwags):
    """
    Returns a HTML image tag containing a base64 encoded sparkline style plot
    """
    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    ax.plot(data)
    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    img = StringIO()
    plt.savefig(img)
    img.seek(0)
    plt.close()
    return '<img src="data:image/png;base64,{}"/>'.format(base64.b64encode(img.read()))

### Random Data Example

In [4]:
n = 100

data = [
    ('rand',        np.random.rand(n)), 
    ('randn',       np.random.randn(n)), 
    ('beta',        np.random.beta(2, 1, size=n)),
    ('binomial',    np.random.binomial(3.4, 0.22, size=n)),
    ('exponential', np.random.exponential(size=n)),
    ('geometric',   np.random.geometric(0.5, size=n)),
    ('laplace',     np.random.laplace(size=n))
]
spark = pd.DataFrame(data, columns=['func', 'data'])
spark

,func,data
0,rand,"[0.708400611586, 0.708496610314, 0.882450351541, ...]"
1,randn,"[0.120334819136, -0.454884770933, 0.0753941935193, ...]"
2,beta,"[0.613852276107, 0.999174876093, 0.200967973268, ...]"
3,binomial,"[0, 0, 0, ...]"
4,exponential,"[2.19486747554, 0.518050303758, 1.13634428504, ...]"
5,geometric,"[2, 2, 2, ...]"
6,laplace,"[0.296682879594, -0.191023864773, -0.0425561055004, ...]"


In [5]:
# map the sparkline function over the data column and store back in the column sparklines
spark['sparklines'] = spark.data.map(sparkline)

# _repr_html_ escapes HTML so manually handle the rendering
HTML(spark.to_html(escape=False))

,func,data,sparklines
0,rand,"[0.708400611586, 0.708496610314, 0.882450351541, ...]",
1,randn,"[0.120334819136, -0.454884770933, 0.0753941935193, ...]",
2,beta,"[0.613852276107, 0.999174876093, 0.200967973268, ...]",
3,binomial,"[0, 0, 0, ...]",
4,exponential,"[2.19486747554, 0.518050303758, 1.13634428504, ...]",
5,geometric,"[2, 2, 2, ...]",
6,laplace,"[0.296682879594, -0.191023864773, -0.0425561055004, ...]",


### Aggregation Example

Query [fixer.io](http://fixer.io/) for currency exchange data against the Euro. Aggregate by country and display sparklines.

In [6]:
def get_fixer_rates(date):
    """
    Query fixer.io for currency exchange data
    """
    res = requests.get("http://api.fixer.io/%s" % date)
    sleep(0.5)  # be kind to the API!
    json = res.json()
    return [(date, json['base'], k, v) for k, v in json['rates'].iteritems()]

In [7]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

dates = daterange(date(2014, 1, 1), date.today())
    
rates = list(chain.from_iterable([get_fixer_rates(date.strftime("%Y-%m-%d")) for date in dates]))

In [8]:
currency_exchange = pd.DataFrame(list(rates), columns=['date', 'base', 'country', 'rates'])
currency_exchange.head()

,date,base,country,rates
0,2014-01-01,EUR,USD,1.3791
1,2014-01-01,EUR,IDR,16764.7800
2,2014-01-01,EUR,BGN,1.9558
3,2014-01-01,EUR,ILS,4.7880
4,2014-01-01,EUR,GBP,0.8337


In [9]:
HTML(currency_exchange.groupby(['country']).aggregate({'rates': sparkline}).to_html(escape=False))

,rates
country,
AUD,
BGN,
BRL,
CAD,
CHF,
CNY,
CZK,
DKK,
GBP,
